## using Azure Table service from Kubernetes
this example was run on azure to show how julia can interact with cloud services.   it set up a channel and workers fetch elements from the channel and then push the results to the azure table service.

In [1]:
using Distributed

In [2]:
workervec = [("dbgannon@104.215.80.203", 2), ("dbgannon@40.74.255.62", 2), ("dbgannon@40.74.246.210", 2),("dbgannon@13.66.4.241", 2)]

4-element Array{Tuple{String,Int64},1}:
 ("dbgannon@104.215.80.203", 2)
 ("dbgannon@40.74.255.62", 2)  
 ("dbgannon@40.74.246.210", 2) 
 ("dbgannon@13.66.4.241", 2)   

In [3]:
addprocs(workervec ; sshflags=`-i /home/dbgannon/.ssh/newkey`, tunnel=true, exename=`/home/dbgannon/julia-1.0.1/bin/julia`)

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [4]:
@everywhere using Distributed

In [6]:
@everywhere using PyCall

In [7]:
@everywhere @pyimport azure.cosmosdb.table as aztable

In [8]:
@everywhere tbl = aztable.TableService("dbgannonstorage", "asure storage key here")

In [9]:
@everywhere function putInTable(table, row, data)
    item = Dict{Any,Any}()
    item["PartitionKey"]= string("Node_",string(myid()))
    item["RowKey"] = string(row)
    item["thedata"] = data
    table[:insert_entity]("datafromjulia", item)
end

In [13]:
x = @spawnat 2 putInTable(tbl, 3,"the data")

Future(2, 1, 129, nothing)

In [14]:
fetch(x)

"W/\"datetime'2018-11-19T01%3A22%3A26.0368565Z'\""

In [15]:
ch = RemoteChannel(()->Channel{String}(1000), 1)

RemoteChannel{Channel{String}}(1, 1, 131)

In [24]:
@everywhere function listen(ch, tbl)
    println("listen started")
    row = 0
    id = "1"
    while id != "-1"
        id = take!(ch)
        x = string("out from ",string(myid())," val=", string(id))
        putInTable(tbl,row, x )
        row = row+1
    end
    println("listen done")
end

In [34]:
x = [@spawnat i listen(ch, tbl) for i in workers()]

8-element Array{Future,1}:
 Future(2, 1, 196, nothing)
 Future(3, 1, 197, nothing)
 Future(4, 1, 198, nothing)
 Future(5, 1, 199, nothing)
 Future(6, 1, 200, nothing)
 Future(7, 1, 201, nothing)
 Future(8, 1, 202, nothing)
 Future(9, 1, 203, nothing)

      From worker 2:	listen started
      From worker 3:	listen started
      From worker 5:	listen started
      From worker 4:	listen started
      From worker 7:	listen started
      From worker 6:	listen started
      From worker 8:	listen started
      From worker 9:	listen started


In [29]:
function putInCh(ch)
    for i=1:8
        put!(ch, string(-1))
    end
    return 1
end

putInCh (generic function with 1 method)

In [35]:
putInCh(ch)

1

      From worker 4:	listen done
      From worker 9:	listen done
      From worker 3:	listen done
      From worker 5:	listen done
      From worker 2:	listen done
      From worker 7:	listen done
      From worker 6:	listen done
      From worker 8:	listen done


In [26]:
put!(ch, string(-1))

RemoteChannel{Channel{String}}(1, 1, 131)

      From worker 3:	listen done


In [31]:
fetch(x)

8-element Array{Future,1}:
 Future(2, 1, 180, nothing)
 Future(3, 1, 181, nothing)
 Future(4, 1, 182, nothing)
 Future(5, 1, 183, nothing)
 Future(6, 1, 184, nothing)
 Future(7, 1, 185, nothing)
 Future(8, 1, 186, nothing)
 Future(9, 1, 187, nothing)

In [23]:
take!(ch)

"-1"